In [11]:
import crystal_toolkit

In [12]:
from ase.io import read

from ase.build import add_adsorbate
from ase.geometry import get_distances
import numpy as np
import random

from pymatgen.core import Structure, Lattice, Molecule
from pymatgen.core.operations import SymmOp

from full_automate import CreateInterface

# Load structure

In [13]:
supercell = Structure.from_file("../_input/quantumatk/POSCAR_sc_li_outer")
# supercell_ase = read("quantumatk/POSCAR_sc_li_outer")
# supercell_ase.cell

molecule = Molecule.from_file("../_input/fg/diurethane.xyz")

# ## alternative
# from pymatgen.io.ase import AseAtomsAdaptor

# molecule_ase = read("fg/triamine.xyz")
# molecule_ase = AseAtomsAdaptor.get_molecule(atoms)

In [14]:
# molecule_centered = molecule.get_centered_molecule()

# molecule_ase_xyz = CreateInterface.align_principal_axes(molecule_ase)
# molecule_ase.get_positions()

# Vary NMC rate

In [15]:
nmc_structure = CreateInterface.substitute_ni_atoms(structure=supercell, nmc_ratio=[1,1,1], seed=None)

Original Ni indices: [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
Shuffled Ni indices: [48, 43, 28, 46, 52, 53, 41, 47, 29, 27, 40, 51, 37, 33, 31, 39, 30, 45, 42, 32, 44, 35, 49, 36, 50, 34, 38]
NMC ratio: [0.33333333 0.33333333 0.33333333]
n_ni: 9, n_mn: 9, n_co: 9


# Step 1: Get longest axis of molecule

In [16]:
molecule_axis = CreateInterface.get_principal_axis(molecule)    # [-0.99961261 -0.02777762  0.00174037]
print(molecule_axis)

# # Step 2: Get molecule's longest axis (in Cartesian)
# molecule_coords = np.array(molecule.cart_coords)
# molecule_extent = np.ptp(molecule_coords, axis=0)
# molecule_axis = molecule_extent / np.linalg.norm(molecule_extent)
# print(molecule_axis)

[-0.99984225 -0.0041892  -0.01726076]


# Step 2: Determine the thickest lattice direction

In [27]:
supercell_frac_coords = np.array([site.frac_coords for site in supercell])
spread = np.ptp(supercell_frac_coords, axis=0)  # peak-to-peak: max - min along each axis
thickest_idx = np.argmax(spread)
lattice_vec_thickest_idx = supercell.lattice.matrix[thickest_idx]

# Get supercell direction (unit vector)
supercell_unitvec = lattice_vec_thickest_idx / np.linalg.norm(lattice_vec_thickest_idx)      # returns: array([-2.65311532e-08,  1.00000000e+00,  0.00000000e+00])

# Step 3: Rotate molecule to align molecule_axis with supercell_unitvec

In [28]:
# Assuming molecule_axis and supercell_unitvec are normalized vectors
rotation_axis = np.cross(molecule_axis, supercell_unitvec)      # array([-0.02777762,  0.99961261,  0.        ])

In [29]:
# Skip rotation if vectors are already aligned
if np.linalg.norm(rotation_axis) > 1e-3:
    rotation_axis /= np.linalg.norm(rotation_axis)
    rotation_angle_rad = np.arccos(np.clip(np.dot(molecule_axis, supercell_unitvec), -1.0, 1.0))    # 1.5690559577350829
    rotation_angle_deg = np.rad2deg(rotation_angle_rad)     # 89.90028419807753

    rotation = SymmOp.from_origin_axis_angle(origin=[0, 0, 0],
                                            axis=rotation_axis,
                                            angle=rotation_angle_deg,
                                            angle_in_radians=False)
    molecule_rotated = molecule.copy()
    molecule_rotated.apply_operation(rotation)
else:
    molecule_rotated = molecule.copy()

# Step 3a: fine-tuning flipping through specific direction

In [ ]:
# Mirror over the y-axis (flip the sign of y-coordinates)
molecule_cart_coords_flipped = molecule_rotated.cart_coords.copy()
# if molecule_name == ["diurethane", "triamine"]:
molecule_cart_coords_flipped[:, 0] *= -1  # Flip x-axis
molecule_cart_coords_flipped[:, 1] *= -1  # Flip y-axis
# molecule_cart_coords_flipped[:, 2] *= -1  # Flip z-axis
# elif molecule_name == "dicarbonate":
# unknown
# elif molecule_name == "triester": # approved
# elif molecule_name == "triether": # approved
# elif molecule_name == "triol": # approved

# Create a new Molecule with flipped coordinates
molecule_rotated_flipped = Molecule(species=molecule_rotated.species, coords=molecule_cart_coords_flipped)
molecule_rotated_flipped

# Step 4: Translate molecule so it sits 3 Å above the top of supercell

## Get longest lattice vector of supercell

In [ ]:
supercell_lengths = supercell.lattice.abc
longest_idx = np.argmax(supercell_lengths)
longest_unitvec = np.array([1 if i == longest_idx else 0 for i in range(3)])

In [ ]:
supercell_cart_coords = np.array([site.coords for site in supercell])
molecule_cart_coords = np.array(molecule_rotated_flipped.cart_coords)

# # --- Get topmost atom in slab along vacuum direction ---
# supercell_top = np.max(supercell_cart_coords[:, longest_idx])

# # Center of the supercell in y++
# supercell_min = np.min(supercell_cart_coords[:, thickest_idx])
# supercell_max = np.max(supercell_cart_coords[:, thickest_idx])
# supercell_center = np.mean(supercell_cart_coords, axis=0)       # didnt really like this
# molecule_center = np.mean(molecule_cart_coords, axis=0)

In [ ]:
# --- Get molecule centroid and shift ---
molecule_com = molecule_rotated_flipped.center_of_mass
molecule_at_origin = molecule_rotated_flipped.translate_sites(range(len(molecule_rotated_flipped)), -molecule_com)

# combined_structure = supercell.copy()
# for site in molecule_at_origin.sites:
#     combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

# combined_structure = combined_structure.get_sorted_structure()

# # Save result
# combined_structure.to(filename="molecule_at_origin_center.cif")

In [ ]:
# supercell center in 2D excluding the longest direction 
# longest_unitvec_inverse = 1 - longest_unitvec   # inversion of longest_unitvec
supercell_2d_center = supercell.lattice.matrix.sum(axis=0) / 2  # Question: how do they compute for y direc?
supercell_2d_center[longest_idx] = 0

In [ ]:
# Place molecule center in 2D center of supercell
molecule_translated_x_y = molecule_at_origin.copy()
translation = supercell_2d_center - molecule_com
molecule_translated_x_y.translate_sites(range(len(molecule_at_origin)), translation)

# combined_structure = supercell.copy()
# for site in molecule_translated_x_y.sites:
#     combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

# combined_structure = combined_structure.get_sorted_structure()

# # Save result
# combined_structure.to(filename="molecule_at_origin_center_translated_x_y.cif")

In [ ]:
# Get z-coordinate of lowest atom in molecule
z_low_molecule = min(site.z for site in molecule_translated_x_y.sites)
# print(z_low_molecule)

In [ ]:
# Then lift molecule along thickest_idx axis to be 3 Å above top

# Get z-coordinate of lowest atom in molecule
z_low_molecule = min(site.z for site in molecule_translated_x_y.sites)

# z_top_supercell = np.max([site.coords[longest_idx] for site in supercell.sites])
z_top_supercell = np.max(supercell_cart_coords[:, longest_idx])
# molecule_bottom = np.min([site[longest_idx] for site in molecule_rotated.cart_coords])
# z_shift = z_top_supercell - molecule_bottom + 3.0  # 3 Å vacuum
translation_z = np.zeros(3)
translation_z[longest_idx] = z_top_supercell - z_low_molecule + 3.0  # 3 Å vacuum

molecule_translated_x_y_z = molecule_translated_x_y.copy()
molecule_translated_x_y_z.translate_sites(range(len(molecule_translated_x_y)), translation_z)

combined_structure = supercell.copy()
for site in molecule_translated_x_y_z.sites:
    combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

combined_structure = combined_structure.get_sorted_structure()

# Save result
combined_structure.to(filename="molecule_at_origin_center_translated_x_y_z.cif")

In [ ]:
# # Then lift molecule along longest axis to be 3 Å above top
# molecule_translated_x_y_z = molecule_translated_x_y.copy()
# offset = np.zeros(3)
# offset[longest_idx] = supercell_max - np.min(molecule_cart_coords[:, longest_idx]) + 3.0
# molecule_translated_x_y_z.translate_sites(range(len(molecule_translated_x_y)), offset)

# print("Note: doesn't work")